In [28]:
import os
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import sys
from joblib import Parallel, delayed
logging.basicConfig(level=logging.ERROR)


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# 
class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            self.model_path = "/kaggle/input/birdclef-2025-baseline-fold0-0404"
            
            self.device = "cpu"
            self.batch_size = 8
            self.n_jobs = 3
            
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
            self.MODELS_DIR = "../models/"
            self.model_path =  "../models/mel_cleaned0413_vino/"
            
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.batch_size = 32
            self.n_jobs = 3

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5
        self.TARGET_DURATION = 5
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 64
        self.N_MELS = 148
        self.FMIN = 20
        self.FMAX = 16000
        
        self.seed = 42
        
        self.smooth_center_weight = 0.6
        self.smooth_neighbor_weight = 0.2
        
        # ===== Inference Mode =====
        if mode == "inference":
            self.use_tta = False
            self.tta_count = 3
            self.threshold = 0.5

            self.use_specific_folds = False
            self.folds = [0, 1, 2, 3, 4]  # Used only if use_specific_folds is True

            self.debug_count = 3
            
            
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

In [30]:
"TODO: configを2つにわけるべきかも．柔軟に変える方はnotebook側で，固定したい方はmodule側とか"
"TODO: Debugモードになっていたらsubmissionでエラーになる"
"TODO: 疑似ラベル作成のためなら，事前にデータセットを準備するべき"
cfg = CFG(mode='inference', kaggle_notebook=False)

if cfg.KAGGLE_NOTEBOOK:
    !pip install -U openvino-telemetry  --no-index --find-links /kaggle/input/pip-hub
    !pip install -U openvino  --no-index --find-links /kaggle/input/pip-hub
    sys.path.append("/kaggle/input/birdclef-2025-libs/")
    
from openvino.runtime import Core
from module import models_lib, utils_lib, preprocess_lib, inference_lib

# Set seed
utils_lib.set_seed(cfg.seed)

In [31]:
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")

Using device: cuda
Loading taxonomy data...
Number of classes: 206


In [32]:
def process_audio_file(audio_path, cfg):
    """1ファイル分のmelspecデータを返す（row_id, melspecのリスト）"""
    dataset = []
    soundscape_id = Path(audio_path).stem
    try:
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
        total_segments = int(len(audio_data) / (cfg.FS * cfg.WINDOW_SIZE))

        for segment_idx in range(total_segments):
            start = int(segment_idx * cfg.FS * cfg.WINDOW_SIZE)
            end = int(start + cfg.FS * cfg.WINDOW_SIZE)
            segment_audio = audio_data[start:end]

            mel_spec = preprocess_lib.process_audio_segment(segment_audio, cfg)
            row_id = f"{soundscape_id}_{(segment_idx + 1) * cfg.WINDOW_SIZE}"

            dataset.append({
                "row_id": row_id,
                "mel_spec": mel_spec
            })
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
    return dataset

def generate_melspec_dataset(cfg):
    test_dir = Path(cfg.test_soundscapes)
    if not test_dir.exists():
        print(f"Test directory {test_dir} does not exist.")
        return []

    test_files = list(test_dir.glob('*.ogg'))
    if len(test_files) == 0:
        print("No test audio files found.")
        return []

    if cfg.debug:
        print(f"Debug mode enabled, using only {cfg.debug_count} files")
        test_files = test_files[:cfg.debug_count]

    results = Parallel(n_jobs=cfg.n_jobs)(
        delayed(process_audio_file)(path, cfg) for path in tqdm(test_files, desc="Parallel melspec gen")
    )
    dataset = [item for sublist in results for item in sublist]
    return dataset

In [33]:
def load_openvino_models(vino_dir, cfg):
    models = []
    vino_dir = Path(vino_dir)

    if cfg.use_specific_folds:
        fold_ids = cfg.folds
        xml_files = [vino_dir / f"model_fold{f}.xml" for f in fold_ids]
    else:
        xml_files = sorted(vino_dir.glob("model_fold*.xml"))

    for xml_path in xml_files:
        bin_path = xml_path.with_suffix(".bin")

        if not xml_path.exists() or not bin_path.exists():
            print(f"⚠️ Warning: Missing files for {xml_path.stem}")
            continue

        core = Core()
        model_ir = core.read_model(xml_path)
        compiled_model = core.compile_model(model_ir, device_name="CPU")
        models.append(compiled_model)

        # 🔍 モデルのファイル名（fold情報）をログに出す
        print(f"✅ Loaded model: {xml_path.name}")

    print(f"🎉 Total {len(models)} OpenVINO model(s) loaded from {vino_dir}")
    return models

# === NEW ===
def run_inference_openvino(dataset, models_ir, cfg, species_ids):
    row_ids = []
    all_preds = []

    for i in range(0, len(dataset), cfg.batch_size):
        batch = dataset[i:i+cfg.batch_size]

        mel_list = [item["mel_spec"] for item in batch]
        input_tensor = np.stack(mel_list).astype(np.float32)  # (B, H, W)
        input_tensor = np.expand_dims(input_tensor, axis=1)  # (B, 1, H, W)

        preds_per_model = []
        for model in models_ir:
            input_layer = model.input(0)
            output_layer = model.output(0)
            result = model([input_tensor])[output_layer]
            probs = 1 / (1 + np.exp(-result))  # sigmoid
            preds_per_model.append(probs)

        avg_preds = np.mean(preds_per_model, axis=0)
        all_preds.append(avg_preds)
        row_ids.extend([item["row_id"] for item in batch])

    predictions = np.concatenate(all_preds, axis=0)
    return row_ids, predictions

In [34]:
def smooth_submission_df(submission_df, cfg):
    """
    Smooth predictions in a submission DataFrame and return the modified DataFrame.

    Each row's prediction is averaged with its neighbors using weights from cfg.

    :param submission_df: DataFrame containing submission data (row_id + prediction columns)
    :param cfg: Configuration object containing smoothing weights
    :return: Smoothed submission DataFrame
    """
    print("Smoothing submission predictions...")
    sub = submission_df.copy()
    cols = sub.columns[1:]
    groups = sub['row_id'].astype(str).str.rsplit('_', n=1).str[0].values
    unique_groups = np.unique(groups)

    for group in unique_groups:
        idx = np.where(groups == group)[0]
        sub_group = sub.iloc[idx].copy()
        predictions = sub_group[cols].values
        new_predictions = predictions.copy()

        if predictions.shape[0] > 1:
            w_c = cfg.smooth_center_weight
            w_n = cfg.smooth_neighbor_weight
            new_predictions[0] = predictions[0] * (1 - w_n) + predictions[1] * w_n
            new_predictions[-1] = predictions[-1] * (1 - w_n) + predictions[-2] * w_n
            for i in range(1, predictions.shape[0] - 1):
                new_predictions[i] = (
                    predictions[i - 1] * w_n +
                    predictions[i] * w_c +
                    predictions[i + 1] * w_n
                )
        sub.iloc[idx, 1:] = new_predictions

    return sub

In [35]:
print("Generating dataset...")
dataset = generate_melspec_dataset(cfg)

print("Loading OpenVINO models...")
vino_dir = Path(cfg.model_path).with_name(Path(cfg.model_path).name)
models_ir = load_openvino_models(vino_dir, cfg)

if not models_ir:
    raise RuntimeError("No OpenVINO models found.")

print("Running OpenVINO inference...")
if len(dataset) > 0:
    row_ids, predictions = run_inference_openvino(dataset, models_ir, cfg, species_ids)
else:
    print("No test data available, generating empty submission.")
    row_ids = []
    predictions = []

submission_df = utils_lib.create_submission(row_ids, predictions, species_ids, cfg)
submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv')
submission_df.to_csv(submission_path, index=False)

print(f"Submission saved to {submission_path}")


Generating dataset...


Parallel melspec gen:   0%|          | 0/71 [00:00<?, ?it/s]

Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
✅ Loaded model: model_fold1.xml
🎉 Total 2 OpenVINO model(s) loaded from ../models/mel_cleaned0413_vino
Running OpenVINO inference...
Creating submission dataframe...
Submission saved to ../data/result/submission_before_smoothing.csv


In [36]:
# Smooth and save separately
smoothed_df = smooth_submission_df(submission_df, cfg)
smoothed_submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission.csv')
smoothed_df.to_csv(smoothed_submission_path, index=False)
print(f"Smoothed submission saved to {smoothed_submission_path}")

Smoothing submission predictions...
Smoothed submission saved to ../data/result/submission.csv


In [37]:
# 提出用ファイルを読み込む
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission.csv'))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.001942,0.001313,0.001430,0.000845,0.003096,0.007352,0.004102,0.002791,0.014801,...,0.005447,0.008643,0.002247,0.000538,0.000256,0.003199,0.000282,0.003936,0.000500,0.000591
1,H02_20230420_074000_10,0.001340,0.000944,0.001165,0.000545,0.002651,0.008540,0.003395,0.001907,0.009437,...,0.005022,0.005416,0.003435,0.000457,0.000584,0.002841,0.000308,0.003416,0.000365,0.000423
2,H02_20230420_074000_15,0.001341,0.001562,0.001111,0.000277,0.003136,0.005459,0.004780,0.001549,0.019664,...,0.003953,0.008091,0.007206,0.000296,0.001314,0.001882,0.000420,0.002167,0.000415,0.000363
3,H02_20230420_074000_20,0.002708,0.003865,0.002321,0.000398,0.007377,0.007488,0.003542,0.002942,0.053902,...,0.003048,0.011587,0.003137,0.000461,0.000577,0.002139,0.000367,0.002644,0.000457,0.000565
4,H02_20230420_074000_25,0.002290,0.002411,0.001930,0.000503,0.005461,0.009116,0.003458,0.002321,0.050243,...,0.003715,0.009092,0.002086,0.000871,0.000317,0.002887,0.000805,0.004089,0.000579,0.000908


In [38]:
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv'))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.002092,0.001393,0.001449,0.000913,0.003071,0.006365,0.004645,0.002988,0.016495,...,0.005523,0.009995,0.002552,0.000543,0.000273,0.003191,0.000295,0.003950,0.000555,0.000639
1,H02_20230420_074000_10,0.001343,0.000993,0.001351,0.000572,0.003197,0.011298,0.001930,0.002003,0.008024,...,0.005144,0.003235,0.001024,0.000517,0.000188,0.003232,0.000230,0.003879,0.000277,0.000402
2,H02_20230420_074000_15,0.000581,0.000346,0.000325,0.000097,0.000593,0.002442,0.006543,0.000541,0.006618,...,0.004157,0.007381,0.011548,0.000193,0.002086,0.001318,0.000554,0.001495,0.000439,0.000271
3,H02_20230420_074000_20,0.003622,0.005779,0.003228,0.000523,0.010705,0.008669,0.002340,0.004121,0.070440,...,0.002149,0.015076,0.000361,0.000387,0.000124,0.002224,0.000210,0.002469,0.000479,0.000599
4,H02_20230420_074000_25,0.002095,0.001643,0.001598,0.000321,0.004178,0.008994,0.004145,0.001803,0.051570,...,0.004635,0.005326,0.003056,0.000951,0.000426,0.002704,0.000650,0.004316,0.000407,0.000758


In [39]:
submission

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.002092,0.001393,0.001449,0.000913,0.003071,0.006365,0.004645,0.002988,0.016495,...,0.005523,0.009995,0.002552,0.000543,0.000273,0.003191,0.000295,0.003950,0.000555,0.000639
1,H02_20230420_074000_10,0.001343,0.000993,0.001351,0.000572,0.003197,0.011298,0.001930,0.002003,0.008024,...,0.005144,0.003235,0.001024,0.000517,0.000188,0.003232,0.000230,0.003879,0.000277,0.000402
2,H02_20230420_074000_15,0.000581,0.000346,0.000325,0.000097,0.000593,0.002442,0.006543,0.000541,0.006618,...,0.004157,0.007381,0.011548,0.000193,0.002086,0.001318,0.000554,0.001495,0.000439,0.000271
3,H02_20230420_074000_20,0.003622,0.005779,0.003228,0.000523,0.010705,0.008669,0.002340,0.004121,0.070440,...,0.002149,0.015076,0.000361,0.000387,0.000124,0.002224,0.000210,0.002469,0.000479,0.000599
4,H02_20230420_074000_25,0.002095,0.001643,0.001598,0.000321,0.004178,0.008994,0.004145,0.001803,0.051570,...,0.004635,0.005326,0.003056,0.000951,0.000426,0.002704,0.000650,0.004316,0.000407,0.000758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,H02_20230502_080500_40,0.000011,0.000007,0.000007,0.000004,0.000012,0.000046,0.000034,0.000010,0.000005,...,0.000384,0.014090,0.000253,0.001887,0.001861,0.000016,0.002484,0.000210,0.000392,0.000070
848,H02_20230502_080500_45,0.000070,0.000019,0.000115,0.000013,0.000070,0.000585,0.000230,0.000090,0.000035,...,0.006423,0.025827,0.003563,0.002110,0.005266,0.002617,0.001573,0.004855,0.015814,0.000994
849,H02_20230502_080500_50,0.000025,0.000012,0.000042,0.000002,0.000017,0.000127,0.000085,0.000024,0.000014,...,0.010732,0.018669,0.001447,0.000713,0.006571,0.000638,0.002715,0.002678,0.001687,0.000142
850,H02_20230502_080500_55,0.000004,0.000003,0.000003,0.000001,0.000006,0.000036,0.000005,0.000004,0.000001,...,0.000273,0.004114,0.000024,0.000883,0.000669,0.000025,0.006597,0.000104,0.000361,0.000043


In [40]:
print("✅ Shape:", submission.shape)
print("✅ Columns:", submission.columns.tolist())
print("✅ Dtypes:\n", submission.dtypes)
print("✅ Nulls:\n", submission.isna().sum().sum())

✅ Shape: (852, 207)
✅ Columns: ['row_id', '1139490', '1192948', '1194042', '126247', '1346504', '134933', '135045', '1462711', '1462737', '1564122', '21038', '21116', '21211', '22333', '22973', '22976', '24272', '24292', '24322', '41663', '41778', '41970', '42007', '42087', '42113', '46010', '47067', '476537', '476538', '48124', '50186', '517119', '523060', '528041', '52884', '548639', '555086', '555142', '566513', '64862', '65336', '65344', '65349', '65373', '65419', '65448', '65547', '65962', '66016', '66531', '66578', '66893', '67082', '67252', '714022', '715170', '787625', '81930', '868458', '963335', 'amakin1', 'amekes', 'ampkin1', 'anhing', 'babwar', 'bafibi1', 'banana', 'baymac', 'bbwduc', 'bicwre1', 'bkcdon', 'bkmtou1', 'blbgra1', 'blbwre1', 'blcant4', 'blchaw1', 'blcjay1', 'blctit1', 'blhpar1', 'blkvul', 'bobfly1', 'bobher1', 'brtpar1', 'bubcur1', 'bubwre1', 'bucmot3', 'bugtan', 'butsal1', 'cargra1', 'cattyr', 'chbant1', 'chfmac1', 'cinbec1', 'cocher1', 'cocwoo1', 'colara1', '